In [2]:
from fake_useragent import UserAgent
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException, TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import json
import logging
import concurrent.futures
import pandas as pd

# Set up logging
logging.basicConfig(level=logging.INFO)

def get_random_user_agent():
    ua = UserAgent()
    return ua.random

def scrape_product(shoe):
    try:
        name_element = WebDriverWait(shoe, 20).until(EC.presence_of_element_located((By.XPATH, ".//span[@class='a-size-base-plus a-color-base a-text-normal']")))
        name = name_element.text
    except (NoSuchElementException, StaleElementReferenceException, TimeoutException):
        logging.exception("Failed to extract product name")
        name = "No name available"

    try:
        product_id = shoe.get_attribute("data-asin")
    except (NoSuchElementException, StaleElementReferenceException, TimeoutException):
        logging.exception("Failed to extract product ID")
        product_id = ""

    try:
        price_element = shoe.find_element(By.XPATH, ".//span[@class='a-price-whole']")
        price = price_element.text if price_element else "0"
    except (NoSuchElementException, StaleElementReferenceException, TimeoutException):
        logging.exception("Failed to extract product price")
        price = "No price available"

    try:
        product_url_element = shoe.find_element(By.XPATH, ".//a[@class='a-link-normal s-no-outline']")
        product_url = product_url_element.get_attribute("href") if product_url_element else ""
    except (NoSuchElementException, StaleElementReferenceException, TimeoutException):
        logging.exception("Failed to extract product URL")
        product_url = ""

    try:
        brand_element = shoe.find_element(By.XPATH, ".//span[@class='a-size-base-plus a-color-base']")
        brand = brand_element.text.strip()
    except (NoSuchElementException, StaleElementReferenceException, TimeoutException):
        logging.exception("Failed to extract product description")
        brand = "No description available"

    return {
        "product_name": name,
        "product_id": product_id,
        "price": price,
        "product_url": product_url,
        "brand_name": brand,
        "gender": "female"  # Hardcoded gender as female
    }

def main():
    # Use the Firefox driver
    options = webdriver.FirefoxOptions()
    driver = webdriver.Firefox(options=options)

    try:
        all_data = []  # List to store data from all pages

        for page_number in range(101, 150):  # Scraping 10 pages
            url = f"https://www.amazon.in/s?k=hand+bags&rh=n%3A1983338031&ref=nb_sb_noss&page={page_number}"
            logging.info(f"Scraping page {page_number}: {url}")

            # Setting random user agent for each request
            user_agent = get_random_user_agent()
            logging.info(f"Using user agent: {user_agent}")
            options.set_preference("general.useragent.override", user_agent)

            driver.get(url)
            driver.maximize_window()

            # Add a delay to allow the page to load
            time.sleep(2)

            # Adjusted XPath expression to capture all product elements
            shoes = driver.find_elements(By.XPATH, '//div[@data-component-type="s-search-result"]')

            data = []

            # Concurrently scrape product information
            with concurrent.futures.ThreadPoolExecutor() as executor:
                results = executor.map(scrape_product, shoes)

            # Collect results
            for result in results:
                data.append(result)
                print(json.dumps(result, indent=4))

            all_data.extend(data)

            logging.info(f"Page {page_number} data collected")

        # Save all data to a single JSON file
        with open("handbags1.json", "w") as json_file:
            json.dump(all_data, json_file, indent=4)
        logging.info("All data saved to amazon_shoes_women.json")

        # Convert all data to a DataFrame
        df = pd.DataFrame(all_data)
        df.to_csv("handbags1.csv", index=False)
        logging.info("All data saved to amazon_shoes_women.csv")

    finally:
        driver.quit()

if __name__ == "__main__":
    main()


INFO:root:Scraping page 101: https://www.amazon.in/s?k=hand+bags&rh=n%3A1983338031&ref=nb_sb_noss&page=101
INFO:root:Using user agent: Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:109.0) Gecko/20100101 Firefox/117.0
INFO:root:Page 101 data collected
INFO:root:Scraping page 102: https://www.amazon.in/s?k=hand+bags&rh=n%3A1983338031&ref=nb_sb_noss&page=102
INFO:root:Using user agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36
INFO:root:Page 102 data collected
INFO:root:Scraping page 103: https://www.amazon.in/s?k=hand+bags&rh=n%3A1983338031&ref=nb_sb_noss&page=103
INFO:root:Using user agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36


{
    "product_name": "SR Glitzy hand Bag",
    "product_id": "B0CSDSDL4N",
    "price": "699",
    "product_url": "https://www.amazon.in/sspa/click?ie=UTF8&spc=MToxNDcxNjkzNjEzNzg1MDUwOjE3MDg0NTY4ODc6c3BfYXRmX25leHQ6MzAwMTMyNDQyOTc4MzMyOjowOjo&url=%2FGeneric-SR-Glitzy-Hand-Bags%2Fdp%2FB0CSDSDL4N%2Fref%3Dsr_1_4849_sspa%3Fdib%3DeyJ2IjoiMSJ9.-Gxm5DON99r7Mo_bjxS74ISKkT6ron5cpjizndGsj9worFcmN9IIVEYdyybLlqs3dMKjO8PPQCfUqBl6KRviI4zT8TCE-Vrqgb9iSSwPoLHhIn0Lxml8wmSAX1Yenztllrmr1UtgHviUCfL2IBR4_TRGCAIVGNWSmcDesInPKmGl8zmuOlVFA6fO27sZefLRn0R0VFNSixXzIH4mZXnRedKI2uIo1xwFyGAWADY0OGrkEQTDqCffwJxwRs2hLppbro-hxZ1UxrXHJwyb40F6UEjuL8vf8JZbdOourknVS1U.20qrtBmWixa-mZQv31-0p3o5AkOw5XrqB-mlHH0yfa8%26dib_tag%3Dse%26keywords%3Dhand%2Bbags%26qid%3D1708456887%26s%3Dshoes%26sr%3D1-4849-spons%26sp_csd%3Dd2lkZ2V0TmFtZT1zcF9hdGZfbmV4dA%26psc%3D1",
    "brand_name": "Generic",
    "gender": "female"
}
{
    "product_name": "Hand Bags",
    "product_id": "B0CSPQGN9T",
    "price": "699",
    "product_url": "https://

INFO:root:Page 103 data collected
INFO:root:Scraping page 104: https://www.amazon.in/s?k=hand+bags&rh=n%3A1983338031&ref=nb_sb_noss&page=104
INFO:root:Using user agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36


{
    "product_name": "Women's Vegan Leather Penguin Handbag (Large)",
    "product_id": "B0C9HQT7TR",
    "price": "695",
    "product_url": "https://www.amazon.in/sspa/click?ie=UTF8&spc=MTo1MDE0MTY2MTAzNTAyMDUwOjE3MDg0NTY5MDU6c3BfYXRmX25leHQ6MzAwMTEyNTAwMjM2NTMyOjowOjo&url=%2FKeywest-Womens-Leather-Penguin-Handbag%2Fdp%2FB0C9HQT7TR%2Fref%3Dsr_1_4897_sspa%3Fdib%3DeyJ2IjoiMSJ9.tNVLl8Kaq6F9KyRShOyWqADcDk9TftxElr5kBSEeLKD5IwitU0edi98JGmmBz7x36BXczO_K8pSHygHf_WU8FZrnH-D_IcSmpvi8-xvO_vRBFE-tCn8aiYA6BzhT73tjXmhqvLvadPPoNpkF5sSwz5pwjkMvYuZB3zHY-ZfFqA2zbgKlIlprTe29kYW8DlgMRnq9owTDcq7jBR11KKRoHUchWokrrUbWfYs9BvvM3LPw3nH8Nf0cqaUZzfhhoPLjkw2pckMKfuqNlBeLyCOQ2dfrZ5KNP5ajuhV8tYMHnvg.htRtubueZl7jgapAFTWcfoSb823NRJS9hLWJmZLd_aM%26dib_tag%3Dse%26keywords%3Dhand%2Bbags%26qid%3D1708456905%26s%3Dshoes%26sr%3D1-4897-spons%26sp_csd%3Dd2lkZ2V0TmFtZT1zcF9hdGZfbmV4dA%26psc%3D1",
    "brand_name": "Keywest",
    "gender": "female"
}
{
    "product_name": "Full-Grain Natural Cro Leather Top-Handle satchel Tote

INFO:root:Page 104 data collected
INFO:root:Scraping page 105: https://www.amazon.in/s?k=hand+bags&rh=n%3A1983338031&ref=nb_sb_noss&page=105
INFO:root:Using user agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36


...

INFO:root:Page 148 data collected
INFO:root:Scraping page 149: https://www.amazon.in/s?k=hand+bags&rh=n%3A1983338031&ref=nb_sb_noss&page=149
INFO:root:Using user agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36 OPR/102.0.0.0


{
    "product_name": "Women's Vegan Leather Vintage Hobo Shoulder Bag (Large)",
    "product_id": "B0CQNZRLNR",
    "price": "795",
    "product_url": "https://www.amazon.in/sspa/click?ie=UTF8&spc=MToxMTc4OTM4ODk4MDIzMDUwOjE3MDg0NTc0NjQ6c3BfYXRmX25leHQ6MzAwMTEyNDgwNDc2NDMyOjowOjo&url=%2FKeywest-Womens-Leather-Vintage-Handbag%2Fdp%2FB0CQNZRLNR%2Fref%3Dsr_1_7057_sspa%3Fdib%3DeyJ2IjoiMSJ9.vmUKzvdUtYu7sWGzZFbS9FXWFf_SOnMcgXJ5xHjSIM0CO1Ioye6r_tZ7wslh6w3bF9fcdXOAwGiu5nKT_22rkoNXx25BhYcFtOH3I2wERf1meNt6YcrCPNlWupLON88A3ZVi3QlTU7sSfMOuVsxYVVJqHq32dlzYc3wEBPy169xIzgink5eB7NOs_vNZEUc2woLWYr425yHrqa1V8jSbJ4n5wri7Vh7wMd4fipp5AU6zCd-IpRDZckPAEEbL2P0-ifPOOW71zQamxOn0yiHr7R5nkcyY3execa_rb8Rxa3I.cYCJFtI_ydqkZ-Jx-e1F7MM-OmADT0A0hZ11soIj6y8%26dib_tag%3Dse%26keywords%3Dhand%2Bbags%26qid%3D1708457464%26s%3Dshoes%26sr%3D1-7057-spons%26sp_csd%3Dd2lkZ2V0TmFtZT1zcF9hdGZfbmV4dA%26psc%3D1",
    "brand_name": "Keywest",
    "gender": "female"
}
{
    "product_name": "ALLAN FEDER Solid Emily Leather Women's Hand

INFO:root:Page 149 data collected
INFO:root:All data saved to amazon_shoes_women.json


{
    "product_name": "Crossbody Sling Bag For Women | Crossbody Bag for Women | Womens PU Leather handbag | Girls Trendy Handbags",
    "product_id": "B0CGJLCHX4",
    "price": "1,499",
    "product_url": "https://www.amazon.in/Deesse-Crossbody-Leather-handbag-Handbags/dp/B0CGJLCHX4/ref=sr_1_7105?dib=eyJ2IjoiMSJ9.kQqh0n45e-AErRxm_vSMMN9U3HDxEf6_udNWfBDUKcKzuYibzhnoeEAmXJc_uLaaKJ0nVNbGl-aMgYrh-UH65ikFbwbQ0cf_BeabMF4NGMvNi1PdbkH8l9evcbijkqkoFW-CfHfrpGKG30gBVMXFmk0wyl0WNZpVQzyCx95mVWUS4AtYWjTBi1zk3WqnT7b2Htg5wJFP-DinyVI1uBGfnqw9pprr7m1zL-ZBlTwyQ2c4ma9NboEFWcrjX_AfuEYN6GsZteaPP0YSdwh0uytnEK3eEFLeVw_oOHeqTf_ofnQ.feCVMkHT2Souk7Z3uGiRb9zcBP0ZS608OgcA5r2A70s&dib_tag=se&keywords=hand+bags&qid=1708457476&s=shoes&sr=1-7105",
    "brand_name": "Deesse",
    "gender": "female"
}
{
    "product_name": "Anm Creations Women's Hand Embroidered Potli Wallet Soft Velvet Fabric Single Potli Bag",
    "product_id": "B07NVH3Y69",
    "price": "299",
    "product_url": "https://www.amazon.in/Creations-Women

INFO:root:All data saved to amazon_shoes_women.csv


In [ ]:
#COMBINING ALL SCRAPPED DATA 

In [2]:
import json

# List of JSON files to combine
file_names =[ 'amazon_shoes_women12.json', 'amazon_shoes_women13.json','watch1.json','amazon_shoes_women15.json','amazon_shoes_women14.json','menshirts1.json','menshirts2.json','menshirts3.json','menshoes1.csv','menshoes2.csv','handbags1.json']

# List to store all data from JSON files
all_data = []

# Iterate over each file and append its data to all_data
for file_name in file_names:
    with open(file_name, 'r') as file:
        data = json.load(file)
        all_data.extend(data)

# Path to save the combined JSON file
output_file = 'scrapped_data.json'

# Save the combined data as a single JSON file
with open(output_file, 'w') as file:
    json.dump(all_data, file, indent=4)

print(f"Combined JSON file saved as {output_file}")


JSONDecodeError: Expecting value: line 1 column 1 (char 0)